# Pre-Process the data
Function: Throw data of each gene to corresponding folder based on their platform (e.g. GPL96, GPL97, GPL8300, etc.)

In [1]:
import GEOparse
import re
import os
import pandas as pd

## Extract Gene Info from GEO DataSets

In [2]:
gsename = "GSE2770"
softfile = "./{}_family.soft.gz".format(gsename)
if os.path.isfile(softfile):
    gse = GEOparse.get_GEO(filepath=softfile)
else:
    gse = GEOparse.get_GEO(geo=gsename, destdir="./")

07-Aug-2019 13:02:02 INFO GEOparse - Parsing ./GSE2770_family.soft.gz: 
07-Aug-2019 13:02:02 DEBUG GEOparse - DATABASE: GeoMiame
07-Aug-2019 13:02:02 DEBUG GEOparse - SERIES: GSE2770
07-Aug-2019 13:02:02 DEBUG GEOparse - PLATFORM: GPL96
07-Aug-2019 13:02:04 DEBUG GEOparse - PLATFORM: GPL97
07-Aug-2019 13:02:05 DEBUG GEOparse - PLATFORM: GPL8300
07-Aug-2019 13:02:05 DEBUG GEOparse - SAMPLE: GSM60348
07-Aug-2019 13:02:05 DEBUG GEOparse - SAMPLE: GSM60349
07-Aug-2019 13:02:05 DEBUG GEOparse - SAMPLE: GSM60350
07-Aug-2019 13:02:05 DEBUG GEOparse - SAMPLE: GSM60351
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60352
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60353
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60354
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60355
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60356
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60357
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GSM60358
07-Aug-2019 13:02:06 DEBUG GEOparse - SAMPLE: GS

## Extract Platform Information

In [9]:
keys = []
values = []
for gpl in gse.gpls:
    keys.append(gse.gpls[gpl].name)
    r1 = re.findall(r"\[.*?\]",gse.gpls[gpl].metadata['title'][0])
    values.append(r1[0][4:-1])
    #values.append(gse.gpls[gpl].metadata['title'][0])
    print(gse.gpls[gpl].name)
    print(gse.gpls[gpl].metadata['title'][0])
    
celformat = dict(zip(keys, values))

GPL96
[HG-U133A] Affymetrix Human Genome U133A Array
GPL97
[HG-U133B] Affymetrix Human Genome U133B Array
GPL8300
[HG_U95Av2] Affymetrix Human Genome U95 Version 2 Array


## Classify the Samples by Platform

In [10]:
keys = []
values = []
for gsm in gse.gsms:
    #print(gse.gsms[gsm].name)
    keys.append(gse.gsms[gsm].name)
    for key,val in celformat.items():
        r1 = re.findall(r"{}".format(val),gse.gsms[gsm].columns.loc['ID_REF','description'])
        if not r1:
            pass
        else:
            values.append(key)
    #r1 = re.findall(r"\(.*?\)",gse.gsms[gsm].columns.loc['ID_REF','description'])
    #values.append(r1[0][1:-1])
    #values.append(gse.gsms[gsm].columns.loc['ID_REF','description'])

gsm_platform = dict(zip(keys, values))
gsm_platform

{'GSM60348': 'GPL8300',
 'GSM60349': 'GPL8300',
 'GSM60350': 'GPL8300',
 'GSM60351': 'GPL8300',
 'GSM60352': 'GPL8300',
 'GSM60353': 'GPL8300',
 'GSM60354': 'GPL8300',
 'GSM60355': 'GPL8300',
 'GSM60356': 'GPL8300',
 'GSM60357': 'GPL8300',
 'GSM60358': 'GPL8300',
 'GSM60359': 'GPL8300',
 'GSM60360': 'GPL8300',
 'GSM60361': 'GPL8300',
 'GSM60362': 'GPL8300',
 'GSM60363': 'GPL8300',
 'GSM60364': 'GPL8300',
 'GSM60365': 'GPL8300',
 'GSM60366': 'GPL8300',
 'GSM60367': 'GPL8300',
 'GSM60368': 'GPL8300',
 'GSM60369': 'GPL8300',
 'GSM60370': 'GPL8300',
 'GSM60371': 'GPL8300',
 'GSM60372': 'GPL8300',
 'GSM60373': 'GPL8300',
 'GSM60374': 'GPL8300',
 'GSM60375': 'GPL8300',
 'GSM60376': 'GPL8300',
 'GSM60377': 'GPL8300',
 'GSM60378': 'GPL8300',
 'GSM60379': 'GPL8300',
 'GSM60380': 'GPL8300',
 'GSM60381': 'GPL8300',
 'GSM60699': 'GPL96',
 'GSM60700': 'GPL96',
 'GSM60701': 'GPL96',
 'GSM60702': 'GPL96',
 'GSM60703': 'GPL96',
 'GSM60704': 'GPL96',
 'GSM60705': 'GPL96',
 'GSM60706': 'GPL96',
 'GSM607

## Create Folders if not exist, Move files to corresponding folders

In [11]:
# Setup paths
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])
datadir = os.path.join(projectdir,'data')
outputdir = os.path.join(projectdir,'output')
gene = 'GSE2770'
genedir = os.path.join(datadir,gene + '_RAW')

In [12]:
# create a folder for each platform
for key in celformat.keys():
    platformdir = os.path.join(genedir,key)
    if not os.path.exists(platformdir):
        os.makedirs(platformdir)
        print('Path created: {}'.format(platformdir))
    else:
        print('Path already exist: {}'.format(platformdir))


Path already exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL96
Path already exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97
Path already exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL8300


In [91]:
# Move Corresponding Cel files to Folders
#onlyfiles = [f for f in os.listdir(genedir) if os.path.isfile(os.path.join(genedir, f))]
onlyfiles = [f for f in os.listdir(genedir) if f.endswith('.gz')]

for file in onlyfiles:
    filelist = file.split('.')
    prefix = filelist[0]
    if prefix in gsm_platform:
        platform = gsm_platform[prefix]
        platformdir = os.path.join(genedir,platform)
        src_path = os.path.join(genedir, file)
        dst_path = os.path.join(platformdir, file)
        os.rename(src_path,dst_path)
        print('Move {} to {}'.format(src_path,dst_path))
        


Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60739.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60739.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60744.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60744.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60727.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL96/GSM60727.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60756.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60756.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60735.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60735.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60748.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60748.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipel

## Download GPL (Gene ID map for Platforms)

In [11]:
platforms = ['GPL96','GPL97','GPL8300']

maps_list = []
gene_maps = pd.DataFrame([],columns=['GPL96','GPL97','GPL8300','ENTREZ_GENE_ID'])
gene_maps.set_index('ENTREZ_GENE_ID',inplace=True)
for platform in platforms:
    temp =gse.gpls[platform].table[['ID','ENTREZ_GENE_ID']]
    # Save to file
    filefullpath = os.path.join(datadir,'{}entrez.csv'.format(platform))
    print(filefullpath)
    temp.to_csv(filefullpath, index=False)
    # Single Table
    temp.dropna(axis=0,inplace=True)
    temp.set_index('ENTREZ_GENE_ID',inplace=True)
    maps_list.append(temp)
    #gene_maps[platform]=temp['ID']
    #gene_maps.merge(temp,right_index=True)

#gene_maps = pd.concat(maps_list, axis=1, sort=False)
#gene_maps

/Users/zhzhao/Dropbox/Helikar/pipelines/data/GPL96entrez.csv


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


/Users/zhzhao/Dropbox/Helikar/pipelines/data/GPL97entrez.csv
/Users/zhzhao/Dropbox/Helikar/pipelines/data/GPL8300entrez.csv


In [7]:
gse.gpls['GPL96'].table

,ID,GB_ACC,SPOT_ID,Species Scientific Name,Annotation Date,Sequence Type,Sequence Source,Target Description,Representative Public ID,Gene Title,Gene Symbol,ENTREZ_GENE_ID,RefSeq Transcript ID,Gene Ontology Biological Process,Gene Ontology Cellular Component,Gene Ontology Molecular Function
0,1007_s_at,U48705,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Huma...,U48705,discoidin domain receptor tyrosine kinase 1 //...,DDR1 /// MIR4640,780 /// 100616237,NM_001202521 /// NM_001202522 /// NM_001202523...,0001558 // regulation of cell growth // inferr...,0005576 // extracellular region // inferred fr...,0000166 // nucleotide binding // inferred from...
1,1053_at,M87338,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,M87338 /FEATURE= /DEFINITION=HUMA1SBU Human re...,M87338,"replication factor C (activator 1) 2, 40kDa",RFC2,5982,NM_001278791 /// NM_001278792 /// NM_001278793...,0000278 // mitotic cell cycle // traceable aut...,0005634 // nucleus // inferred from electronic...,0000166 // nucleotide binding // inferred from...
2,117_at,X51757,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,X51757 /FEATURE=cds /DEFINITION=HSP70B Human h...,X51757,heat shock 70kDa protein 6 (HSP70B'),HSPA6,3310,NM_002155,0000902 // cell morphogenesis // inferred from...,0005737 // cytoplasm // inferred from direct a...,0000166 // nucleotide binding // inferred from...
3,121_at,X69699,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens...,X69699,paired box 8,PAX8,7849,NM_003466 /// NM_013951 /// NM_013952 /// NM_0...,0001655 // urogenital system development // in...,0005634 // nucleus // inferred from direct ass...,0000979 // RNA polymerase II core promoter seq...
4,1255_g_at,L36861,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,L36861 /FEATURE=expanded_cds /DEFINITION=HUMGC...,L36861,guanylate cyclase activator 1A (retina),GUCA1A,2978,NM_000409 /// XM_006715073,0007165 // signal transduction // non-traceabl...,0001750 // photoreceptor outer segment // infe...,0005509 // calcium ion binding // inferred fro...
5,1294_at,L13852,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,L13852 /FEATURE= /DEFINITION=HUME1URP Homo sap...,L13852,microRNA 5193 /// ubiquitin-like modifier acti...,MIR5193 /// UBA7,7318 /// 100847079,NM_003335 /// NR_049825 /// XM_005265430 /// X...,0006464 // cellular protein modification proce...,0005634 // nucleus // not recorded /// 0005829...,0000166 // nucleotide binding // inferred from...
6,1316_at,X55005,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,X55005 /FEATURE=mRNA /DEFINITION=HSCERBAR Homo...,X55005,"thyroid hormone receptor, alpha",THRA,7067,NM_001190918 /// NM_001190919 /// NM_003250 //...,0000122 // negative regulation of transcriptio...,0000790 // nuclear chromatin // inferred from ...,0000978 // RNA polymerase II core promoter pro...
7,1320_at,X79510,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,X79510 /FEATURE=cds /DEFINITION=HSPTPD1 H.sapi...,X79510,"protein tyrosine phosphatase, non-receptor typ...",PTPN21,11099,NM_007039 /// XM_005267287 /// XM_006720011,0006470 // protein dephosphorylation // tracea...,0005737 // cytoplasm // inferred from electron...,0004721 // phosphoprotein phosphatase activity...
8,1405_i_at,M21121,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,M21121 /FEATURE= /DEFINITION=HUMTCSM Human T c...,M21121,chemokine (C-C motif) ligand 5,CCL5,6352,NM_001278736 /// NM_002985,0000165 // MAPK cascade // inferred from mutan...,0005576 // extracellular region // traceable a...,0004435 // phosphatidylinositol phospholipase ...
9,1431_at,J02843,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,J02843 /FEATURE=cds /DEFINITION=HUMCYPIIE Huma...,J02843,"cytochrome P450, family 2, subfamily E, po

In [68]:
#dir(gse.gpls['GPL96'])
# dir(gse)
#dir(gse.gsms['GSM60749'])
#temp = gse.gsms['GSM60349'].table
#print(temp.VALUE[0])
#dir(gse.gsms['GSM60349'])
temp = gse.gpls['GPL96'].table
temp[['ID','ENTREZ_GENE_ID']]

,ID,ENTREZ_GENE_ID
0,1007_s_at,780 /// 100616237
1,1053_at,5982
2,117_at,3310
3,121_at,7849
4,1255_g_at,2978
5,1294_at,7318 /// 100847079
6,1316_at,7067
7,1320_at,11099
8,1405_i_at,6352
9,1431_at,1571


## Create the big table by combining gsms

In [66]:
# gsm_tables['GPL8300']
# temp.dropna(subset=['ID'],how='any',inplace=True)
# temp
# print(key)
# gse.gsms[key].table
maps_list[0]

,ID
ENTREZ_GENE_ID,
780 /// 100616237,1007_s_at
5982,1053_at
3310,117_at
7849,121_at
2978,1255_g_at
7318 /// 100847079,1294_at
7067,1316_at
11099,1320_at
6352,1405_i_at


In [71]:
# create 3 tables by platform

# initialize with platform
gsm_tables = {}
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    temp = gse.gpls[key].table
    df = temp[['ID','ENTREZ_GENE_ID']]
    df.set_index('ID',inplace=True)
    df.drop_duplicates(keep='last',inplace=True)
    gsm_tables[key] = df

# fill each table
for key,val in gsm_platform.items():
    temp = gse.gsms[key].table.copy()
    temp.rename(columns={"ID_REF": "ID"},inplace=True)
    temp.dropna(subset=['ID'],how='any',inplace=True)
    temp.set_index('ID',inplace=True)
    #col1,col2,col3 = '{}.VALUE'.format(key), '{}.ABS_CALL'.format(key), '{}.DETECTION P-VALUE'.format(key)
    col1,col2,col3 = '{}.CEL.gz'.format(key), '{}.CEL.gz.1'.format(key), '{}.CEL.gz.2'.format(key)
    #gsm_tables[val].loc[:,[col1,col2,col3]] = temp[['VALUE','ABS_CALL','DETECTION P-VALUE']]
    # Sort by VALUE and drop duplicated ID
    temp['ENTREZ_GENE_ID']=gsm_tables[val]['ENTREZ_GENE_ID']
    temp.sort_values(by=['ENTREZ_GENE_ID', 'VALUE'],inplace=True)
    temp.drop_duplicates(subset=['ENTREZ_GENE_ID'],keep='last',inplace=True)
    gsm_tables[val][col1] = temp['VALUE']
    gsm_tables[val][col2] = temp['ABS_CALL']
    gsm_tables[val][col3] = temp['DETECTION P-VALUE']

# dropna
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    gsm_tables[key].dropna(subset=['ENTREZ_GENE_ID'],inplace=True)
    gsm_tables[key].set_index('ENTREZ_GENE_ID',inplace=True)
    # gsm_tables[key].dropna(how='all',inplace=True)
    # gsm_tables[key].drop_duplicates(keep='last',inplace=True)


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [73]:
df_outer = None
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    print('{}: {}'.format(key,gsm_tables[key].shape))
    if df_outer is None:
        df_outer = gsm_tables[key]
    else:
        df_outer = pd.merge(df_outer,gsm_tables[key],on='ENTREZ_GENE_ID', how='outer')

df_outer.dropna(how='all',inplace=True)
print('full : {}'.format(df_outer.shape))
df_outer.sort_index(inplace=True)
df_outer.to_csv(os.path.join(genedir,'{}_full_table.csv'.format(gsename)))
df_outer

GPL96: (13244, 102)
GPL97: (10562, 102)
GPL8300: (9131, 102)
full : (19585, 306)


,GSM60699.CEL.gz,GSM60699.CEL.gz.1,GSM60699.CEL.gz.2,GSM60700.CEL.gz,GSM60700.CEL.gz.1,GSM60700.CEL.gz.2,GSM60701.CEL.gz,GSM60701.CEL.gz.1,GSM60701.CEL.gz.2,GSM60702.CEL.gz,...,GSM60378.CEL.gz.2,GSM60379.CEL.gz,GSM60379.CEL.gz.1,GSM60379.CEL.gz.2,GSM60380.CEL.gz,GSM60380.CEL.gz.1,GSM60380.CEL.gz.2,GSM60381.CEL.gz,GSM60381.CEL.gz.1,GSM60381.CEL.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1.2,A,0.870361,2.2,A,0.805420,1.8,A,0.828613,1.1,...,0.926170,1.6,A,0.824011,8.2,A,0.150487,1.3,A,0.939581
100,139.1,P,0.037598,59.5,A,0.067627,14.3,A,0.194580,73.9,...,0.000388,31.5,P,0.003067,27.9,P,0.000266,41.6,P,0.000266
1000,9.5,A,0.466064,9.0,A,0.303711,7.1,A,0.398926,5.3,...,0.986908,0.6,A,0.795978,0.2,A,0.837065,0.8,A,0.660442
10000,17.2,A,0.067627,14.0,A,0.171387,8.2,A,0.149658,6.5,...,0.602006,13.3,A,0.500000,12.7,A,0.438361,14.9,A,0.302547
100009676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,35.6,P,0.001953,48.7,P,0.000244,40.6,P,0.001221,46.6,...,0.000805,29.0,P,0.000805,25.9,P,0.000468,29.4,P,0.000219
10002,15.7,A,0.129639,11.4,P,0.023926,4.8,A,0.398926,11.6,...,0.397994,6.2,A,0.418069,14.2,A,0.438361,16.2,A,0.358690
10003,1.2,A,0.805420,5.7,A,0.601074,1.0,A,0.828613,1.0,...,0.397994,3.6,A,0.302547,0.8,A,0.849473,1.0,A,0.918663


## End of Code

In [114]:
df_outer.to_csv(os.path.join(genedir,'{}_full_table.csv'.format(gsename)))

In [112]:
datadir

'/Users/zhzhao/Dropbox/Helikar/pipelines/data'

In [103]:
emptydf = pd.DataFrame([],index=['ENTREZ_GENE_ID'])
result = pd.merge(emptydf,gsm_tables['GPL97'], on='ENTREZ_GENE_ID',how='outer')
result

KeyError: 'ENTREZ_GENE_ID'

In [90]:
# Combine 3 tables

# gsm_tables['GPL8300']
# gsm_tables['GPL96']
# gsm_tables['GPL97']
df_outer = pd.merge(gsm_tables['GPL8300'], gsm_tables['GPL96'], on='ENTREZ_GENE_ID', how='outer')
df_outer = pd.merge(df_outer,gsm_tables['GPL97'],on='ENTREZ_GENE_ID', how='outer')
df_outer.dropna(how='all',inplace=True)
df_outer

,GSM60348.CEL.gz,GSM60348.CEL.gz.1,GSM60348.CEL.gz.2,GSM60349.CEL.gz,GSM60349.CEL.gz.1,GSM60349.CEL.gz.2,GSM60350.CEL.gz,GSM60350.CEL.gz.1,GSM60350.CEL.gz.2,GSM60351.CEL.gz,...,GSM60763.CEL.gz.2,GSM60764.CEL.gz,GSM60764.CEL.gz.1,GSM60764.CEL.gz.2,GSM60765.CEL.gz,GSM60765.CEL.gz.1,GSM60765.CEL.gz.2,GSM60766.CEL.gz,GSM60766.CEL.gz.1,GSM60766.CEL.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
5595,118.5,P,0.003585,66.7,P,0.011447,37.7,P,0.035163,63.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7075,8.9,A,0.189687,7.3,A,0.320830,2.8,A,0.284747,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557,1.6,A,0.697453,1.2,A,0.926170,5.3,A,0.358690,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557,5.8,A,0.581931,7.5,A,0.621816,3.1,A,0.621816,2.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,4.1,A,0.810313,4.7,A,0.765443,4.9,A,0.621816,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,4.1,A,0.810313,4.7,A,0.765443,4.9,A,0.621816,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,20.6,A,0.150527,27.7,A,0.218983,35.1,P,0.011447,25.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,20.6,A,0.150527,27.7,A,0.218983,35.1,P,0.011447,25.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,1.5,A,0.765443,1.7,A,0.926170,1.0,A,0.901946,2.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# onlyfiles = [f for f in os.listdir(genedir) if os.path.isfile(os.path.join(genedir, f))]
# onlyfiles
# os.listdir(genedir)
onlyfiles = [f for f in os.listdir(genedir) if f.endswith('.gz')]
onlyfiles

['GSM60739.CEL.gz',
 'GSM60744.CEL.gz',
 'GSM60727.CEL.gz',
 'GSM60756.CEL.gz',
 'GSM60735.CEL.gz',
 'GSM60748.CEL.gz',
 'GSM60760.CEL.gz',
 'GSM60703.CEL.gz',
 'GSM60711.CEL.gz',
 'GSM60351.cel.gz',
 'GSM60379.CEL.gz',
 'GSM60367.CEL.gz',
 'GSM60375.CEL.gz',
 'GSM60353.cel.gz',
 'GSM60377.CEL.gz',
 'GSM60369.CEL.gz',
 'GSM60365.CEL.gz',
 'GSM60737.CEL.gz',
 'GSM60729.CEL.gz',
 'GSM60754.CEL.gz',
 'GSM60725.CEL.gz',
 'GSM60758.CEL.gz',
 'GSM60746.CEL.gz',
 'GSM60713.CEL.gz',
 'GSM60701.CEL.gz',
 'GSM60762.CEL.gz',
 'GSM60361.CEL.gz',
 'GSM60699.CEL.gz',
 'GSM60373.CEL.gz',
 'GSM60357.CEL.gz',
 'GSM60349.CEL.gz',
 'GSM60705.CEL.gz',
 'GSM60766.CEL.gz',
 'GSM60380.CEL.gz',
 'GSM60717.CEL.gz',
 'GSM60709.CEL.gz',
 'GSM60721.CEL.gz',
 'GSM60742.CEL.gz',
 'GSM60733.CEL.gz',
 'GSM60750.CEL.gz',
 'GSM60715.CEL.gz',
 'GSM60764.CEL.gz',
 'GSM60719.CEL.gz',
 'GSM60707.CEL.gz',
 'GSM60752.CEL.gz',
 'GSM60731.CEL.gz',
 'GSM60740.CEL.gz',
 'GSM60723.CEL.gz',
 'GSM60371.CEL.gz',
 'GSM60363.CEL.gz',


In [55]:
# dir(gse.gsms[gsm])
# print(gse.gsms[gsm].metadata['title'])
print(gse.gsms[gsm].columns.loc['ID_REF','description'])
r1 = re.findall(r"U133B",gse.gsms[gsm].columns.loc['ID_REF','description'])
r1

Affymetrix U95Av2 probe ID


[]

In [68]:
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])

In [71]:
gene = 'GSE2770'
genedir = os.path.join(projectdir,gene + '_RAW')
genedir

'/Users/zhzhao/Dropbox/Helikar/pipelines/GSE2770_RAW'

In [39]:
gse.gsms['GSM60349'].table

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,100_g_at,82.0,M,0.054470
1,1000_at,66.7,P,0.011447
2,1001_at,7.3,A,0.320830
3,1002_f_at,1.2,A,0.926170
4,1003_s_at,4.7,A,0.765443
5,1004_at,27.7,A,0.218983
6,1005_at,48.6,P,0.014937
7,1006_at,8.5,A,0.267463
8,1007_s_at,71.8,P,0.017001
9,1008_f_at,183.3,P,0.001602


In [30]:
r1 = re.findall(r"\[.*?\]",celformat['GPL96'])

In [35]:
r1[0][4:-1]

'U133A'

In [3]:
import rpy2
print(rpy2.__version__)

3.0.5


In [4]:
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

pandas2ri.activate()

Converting Rpy2 dataframe to pandas

https://rpy2.github.io/doc/latest/html/pandas.html

In [15]:
utils = importr("utils")
utils.data

R object with classes: ('function',) mapped to:

In [5]:
affy = importr("affy")
string = """
readaffydir <- function(addr){
    crd <- getwd()
    setwd(addr)
    mydata = ReadAffy()
    setwd(crd)
    eset = mas5(mydata)
    eset_PMA <- mas5calls(mydata)
    y <- data.frame(exprs(eset), exprs(eset_PMA), assayDataElement(eset_PMA, "se.exprs"))
    y <- y[,sort(names(y))]
    return(y)
}
"""

affyio = SignatureTranslatedAnonymousPackage(string, "affyio")


In [76]:
# initialize with platform
gsm_maps = {}
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    temp = gse.gpls[key].table.copy()
    df = temp[['ID','ENTREZ_GENE_ID']]
    df.set_index('ID',inplace=True)
    #df.drop_duplicates(keep='last',inplace=True)
    gsm_maps[key] = df

In [14]:
gsm_tables_sc500 = {}
for key in celformat.keys():
    platformdir = os.path.join(genedir,key)
    print('Affy Read Path: {}'.format(platformdir))
    if os.path.exists(platformdir):
        outputdf = affyio.readaffydir(platformdir)
    else:
        print('Path not exist: {}'.format(platformdir))
    outputdf['ENTREZ_GENE_ID'] = gsm_maps[key]['ENTREZ_GENE_ID']
    gsm_tables_sc500[key] = outputdf

background correction: mas 
PM/MM correction : mas 
expression values: mas 
background correcting...

R[write to console]: installing the source package ‘hgu133acdf’


R[write to console]: trying URL 'https://bioconductor.org/packages/3.9/data/annotation/src/contrib/hgu133acdf_2.18.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1732200 bytes (1.7 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to

done.
22283 ids to be processed
|                    |
|####################|
Getting probe level data...
Computing p-values
Making P/M/A Calls
Read Path: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL96
background correction: mas 
PM/MM correction : mas 
expression values: mas 
background correcting...

R[write to console]: installing the source package ‘hgu133bcdf’


R[write to console]: trying URL 'https://bioconductor.org/packages/3.9/data/annotation/src/contrib/hgu133bcdf_2.18.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1724221 bytes (1.6 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to

done.
22645 ids to be processed
|                    |
|####################|
Getting probe level data...
Computing p-values
Making P/M/A Calls
Read Path: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97
background correction: mas 
PM/MM correction : mas 
expression values: mas 
background correcting...done.
12625 ids to be processed
|                    |
|####################|
Getting probe level data...
Computing p-values
Making P/M/A Calls
Read Path: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL8300


In [101]:
# Read files
# gpl8300rawdir = '/Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL8300/'
# outputdf = affyio.readaffydir(gpl8300rawdir)
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    try:
        gsm_tables_sc500[key].dropna(subset=['ENTREZ_GENE_ID'],inplace=True)
        gsm_tables_sc500[key].set_index('ENTREZ_GENE_ID',inplace=True)
    except:
        pass

# Merge table to one
df_outer_sc500 = None
for key,val in celformat.items():
    #df = pd.DataFrame([],columns=['ID','ENTREZ_GENE_ID'])
    print('{}: {}'.format(key,gsm_tables[key].shape))
    if df_outer_sc500 is None:
        df_outer_sc500 = gsm_tables_sc500[key]
    else:
        df_outer_sc500 = pd.merge(df_outer_sc500,gsm_tables_sc500[key],on='ENTREZ_GENE_ID', how='outer')

df_outer_sc500.dropna(how='all',inplace=True)
print('Full: {}'.format(df_outer.shape))
df_outer_sc500.rename(str.lower, axis='columns',inplace=True)
df_outer_sc500

GPL96: (13244, 102)
GPL97: (10562, 102)
GPL8300: (9131, 102)
Full: (19585, 306)


,gsm60699.cel.gz,gsm60699.cel.gz.1,gsm60699.cel.gz.2,gsm60700.cel.gz,gsm60700.cel.gz.1,gsm60700.cel.gz.2,gsm60701.cel.gz,gsm60701.cel.gz.1,gsm60701.cel.gz.2,gsm60702.cel.gz,...,gsm60378.cel.gz.2,gsm60379.cel.gz,gsm60379.cel.gz.1,gsm60379.cel.gz.2,gsm60380.cel.gz,gsm60380.cel.gz.1,gsm60380.cel.gz.2,gsm60381.cel.gz,gsm60381.cel.gz.1,gsm60381.cel.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,334.704077,P,0.009985,558.811658,P,0.014937,819.281546,P,0.001892,500.993724,...,0.003067,885.906655,P,0.002617,769.702265,P,0.001892,805.025121,P,0.000959
780 /// 100616237,334.704077,P,0.009985,558.811658,P,0.014937,819.281546,P,0.001892,500.993724,...,0.004863,368.572648,P,0.000959,483.005555,P,0.002617,389.161130,P,0.001141
780 /// 100616237,341.091529,A,0.123873,328.690266,A,0.106612,445.493664,M,0.045580,291.285615,...,0.003067,885.906655,P,0.002617,769.702265,P,0.001892,805.025121,P,0.000959
780 /// 100616237,341.091529,A,0.123873,328.690266,A,0.106612,445.493664,M,0.045580,291.285615,...,0.004863,368.572648,P,0.000959,483.005555,P,0.002617,389.161130,P,0.001141
780 /// 100616237,454.248970,P,0.008184,375.665366,P,0.013116,439.951453,P,0.003823,319.379501,...,0.003067,885.906655,P,0.002617,769.702265,P,0.001892,805.025121,P,0.000959
780 /// 100616237,454.248970,P,0.008184,375.665366,P,0.013116,439.951453,P,0.003823,319.379501,...,0.004863,368.572648,P,0.000959,483.005555,P,0.002617,389.161130,P,0.001141
780 /// 100616237,526.182671,P,0.013116,550.314222,P,0.025230,667.411790,P,0.004963,638.050161,...,0.003067,885.906655,P,0.002617,769.702265,P,0.001892,805.025121,P,0.000959
780 /// 100616237,526.182671,P,0.013116,550.314222,P,0.025230,667.411790,P,0.004963,638.050161,...,0.004863,368.572648,P,0.000959,483.005555,P,0.002617,389.161130,P,0.001141
5982,176.916009,P,0.039365,142.030594,A,0.073830,66.947377,A,0.284747,145.951413,...,0.013092,208.844821,P,0.039365,179.283658,M,0.043968,229.321354,P,0.021866


In [109]:
df_clean_sc500 = pd.DataFrame([],index=df_outer_sc500.index)
df_clean_sc500 = df_clean_sc500[~df_clean_sc500.index.duplicated(keep='first')]
# df_outer_sc500.rename(str.lower, axis='columns',inplace=True)
for key,val in gsm_platform.items():
    key_low = key.lower()
    col1,col2,col3 = '{}.cel.gz'.format(key_low), '{}.cel.gz.1'.format(key_low), '{}.cel.gz.2'.format(key_low)
    try:
        temp = df_outer_sc500.loc[:,[col1,col2,col3]]
    except:
        print('{} not in df_outer_sc500'.format(key))
        continue
    temp.sort_values(by=['ENTREZ_GENE_ID',col1],inplace=True)
    temp = temp[~temp.index.duplicated(keep='last')]
    df_clean_sc500[col1] = temp[col1]
    df_clean_sc500[col2] = temp[col2]
    df_clean_sc500[col3] = temp[col3]

df_clean_sc500.to_csv(os.path.join(genedir,'{}_sc500_full_table.csv'.format(gsename)))
df_clean_sc500

GSM60348 not in df_outer_sc500


,gsm60349.cel.gz,gsm60349.cel.gz.1,gsm60349.cel.gz.2,gsm60350.cel.gz,gsm60350.cel.gz.1,gsm60350.cel.gz.2,gsm60351.cel.gz,gsm60351.cel.gz.1,gsm60351.cel.gz.2,gsm60352.cel.gz,...,gsm60763.cel.gz.2,gsm60764.cel.gz,gsm60764.cel.gz.1,gsm60764.cel.gz.2,gsm60765.cel.gz,gsm60765.cel.gz.1,gsm60765.cel.gz.2,gsm60766.cel.gz,gsm60766.cel.gz.1,gsm60766.cel.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,717.599098,P,0.017000,966.038612,P,0.003067,724.167413,P,0.005643,910.941864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,140.340345,A,0.204022,124.764102,A,0.107301,90.986765,A,0.250724,145.778367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,68.989968,A,0.458816,55.205940,A,0.302547,97.283915,A,0.175989,47.671811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1281.802247,P,0.017000,902.213482,P,0.019303,919.399037,P,0.027860,783.427483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,200.536466,A,0.218983,277.363337,A,0.060419,300.050853,A,0.117161,153.499853,...,0.143002,373.110629,A,0.164032,626.187096,A,0.186972,401.599221,A,0.186972
7318 /// 100847079,540.953164,P,0.021866,255.370653,P,0.031336,506.635753,P,0.013092,223.252516,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,223.734251,P,0.027860,158.550443,P,0.021866,189.557546,A,0.189687,164.631662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,65.106109,A,0.541184,64.526428,A,0.602006,49.510810,A,0.602006,10.100629,...,0.570568,90.043163,A,0.535424,114.817064,A,0.605162,89.731430,A,0.638946
6352,217.342576,P,0.009985,285.981770,P,0.000388,214.198185,P,0.035163,181.918560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# ratio = pd.DataFrame([],index=df_outer_sc500.index)
# # ratio
# if '.gz.' in 'GSM60699.CEL.gz':
#     print('Y')
# else:
#     print('N')
df_outer_sc500.sort_index(inplace=True)
df_outer_sc50=df_outer.sort_index()
df_outer_sc50

,GSM60699.CEL.gz,GSM60699.CEL.gz.1,GSM60699.CEL.gz.2,GSM60700.CEL.gz,GSM60700.CEL.gz.1,GSM60700.CEL.gz.2,GSM60701.CEL.gz,GSM60701.CEL.gz.1,GSM60701.CEL.gz.2,GSM60702.CEL.gz,...,GSM60378.CEL.gz.2,GSM60379.CEL.gz,GSM60379.CEL.gz.1,GSM60379.CEL.gz.2,GSM60380.CEL.gz,GSM60380.CEL.gz.1,GSM60380.CEL.gz.2,GSM60381.CEL.gz,GSM60381.CEL.gz.1,GSM60381.CEL.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1.2,A,0.870361,2.2,A,0.805420,1.8,A,0.828613,1.1,...,0.926170,1.6,A,0.824011,8.2,A,0.150487,1.3,A,0.939581
100,139.1,P,0.037598,59.5,A,0.067627,14.3,A,0.194580,73.9,...,0.000388,31.5,P,0.003067,27.9,P,0.000266,41.6,P,0.000266
100,139.1,P,0.037598,59.5,A,0.067627,14.3,A,0.194580,73.9,...,0.000219,66.5,P,0.000219,70.1,P,0.000266,94.7,P,0.000322
100,169.0,P,0.001221,62.1,P,0.000244,27.6,A,0.067627,92.3,...,0.000388,31.5,P,0.003067,27.9,P,0.000266,41.6,P,0.000266
100,169.0,P,0.001221,62.1,P,0.000244,27.6,A,0.067627,92.3,...,0.000219,66.5,P,0.000219,70.1,P,0.000266,94.7,P,0.000322
1000,3.0,A,0.567627,9.2,A,0.432373,1.1,A,0.665527,0.6,...,0.081337,7.3,A,0.234557,17.1,P,0.039365,15.2,P,0.014937
1000,3.0,A,0.567627,9.2,A,0.432373,1.1,A,0.665527,0.6,...,0.986908,0.6,A,0.795978,0.2,A,0.837065,0.8,A,0.660442
1000,9.5,A,0.466064,9.0,A,0.303711,7.1,A,0.398926,5.3,...,0.081337,7.3,A,0.234557,17.1,P,0.039365,15.2,P,0.014937


In [107]:
key = 'GSM60349'
key_low = key.lower()
col1,col2,col3 = '{}.cel.gz'.format(key_low), '{}.cel.gz.1'.format(key_low), '{}.cel.gz.2'.format(key_low)
temp = df_outer_sc500.loc[:,[col1,col2,col3]]
temp.sort_values(by=['ENTREZ_GENE_ID',col1],inplace=True)
temp

,gsm60349.cel.gz,gsm60349.cel.gz.1,gsm60349.cel.gz.2
ENTREZ_GENE_ID,,,
1,NaN,NaN,NaN
10,8.070748,A,0.992457
100,377.659443,P,0.004180
100,377.659443,P,0.004180
100,859.767277,P,0.000219
100,859.767277,P,0.000219
1000,5.772804,A,0.910595
1000,5.772804,A,0.910595
1000,80.326726,A,0.162935


In [118]:
df_clean = df_outer.rename(str.lower,axis='columns')

ratio = pd.DataFrame([],index=df_clean_sc500.index)
value_sc50 = pd.DataFrame([],index=df_clean.index)
value_sc500 = pd.DataFrame([],index=df_clean_sc500.index)
abs_call_sc50 = pd.DataFrame([],index=df_clean.index)
abs_call_sc500 = pd.DataFrame([],index=df_clean_sc500.index)
pval_sc50 = pd.DataFrame([],index=df_clean.index)
pval_sc500 = pd.DataFrame([],index=df_clean_sc500.index)

cols = list(df_clean_sc500)
for col in cols:
    if '.gz.1' in col:
        #print(col)
        abs_call_sc500[col.lower()] = df_clean_sc500[col]
    elif '.gz.2' in col:
        pval_sc500[col.lower()] = df_clean_sc500[col]
    else:
        value_sc500[col.lower()] = df_clean_sc500[col]

cols = list(df_clean)
for col in cols:
    if '.gz.1' in col:
        #print(col)
        abs_call_sc50[col.lower()] = df_clean[col]
    elif '.gz.2' in col:
        pval_sc50[col.lower()] = df_clean[col]
    else:
        value_sc50[col.lower()] = df_clean[col]

ratio=value_sc500/value_sc50
ratio2 = abs_call_sc50.equals(abs_call_sc500)
ratio3 = pval_sc500/pval_sc50

In [122]:
ratio

,gsm60348.cel.gz,gsm60349.cel.gz,gsm60350.cel.gz,gsm60351.cel.gz,gsm60352.cel.gz,gsm60353.cel.gz,gsm60354.cel.gz,gsm60355.cel.gz,gsm60356.cel.gz,gsm60357.cel.gz,...,gsm60757.cel.gz,gsm60758.cel.gz,gsm60759.cel.gz,gsm60760.cel.gz,gsm60761.cel.gz,gsm60762.cel.gz,gsm60763.cel.gz,gsm60764.cel.gz,gsm60765.cel.gz,gsm60766.cel.gz
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.986518,9.961995,10.007783,9.983975,9.995464,9.993770,10.007302,9.987846,10.001857,9.978009
10,NaN,10.088435,10.340418,9.975143,8.873164,10.030768,9.719683,9.761515,9.857945,9.531424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,22.745166,28.284069,27.189041,18.372780,22.709987,15.851003,29.394968,14.906679,32.493282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,133.877877,48.869876,35.366414,612.052069,1079.238202,220.367476,182.861217,98.586059,59.017063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,9.761072,9.951911,10.053886,10.013381,10.012372,9.996005,9.952439,9.994622,10.179899,...,12.935154,12.831250,9.995988,31.228219,15.287648,13.054058,25.268909,31.356202,20.534179,20.496009
100009676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.280846,23.996883,39.080741,51.708010,144.654930,9.988133,9.975244,65.425918,9.985330,9.978863
10001,NaN,10.975451,15.911774,15.464025,11.717263,13.660991,11.948121,9.997547,11.356036,16.371531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,9.999679,9.976006,9.979932,9.989131,9.980086,9.978145,9.990132,9.990285,9.981440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,9.036637,9.950216,9.965453,10.215852,10.047174,10.364503,10.067253,10.121659,10.097376,...,9.995350,10.002020,9.995143,10.007252,9.976829,9.985985,9.986452,9.998078,10.003099,10.010246


In [121]:
ratio3

,gsm60348.cel.gz.2,gsm60349.cel.gz.2,gsm60350.cel.gz.2,gsm60351.cel.gz.2,gsm60352.cel.gz.2,gsm60353.cel.gz.2,gsm60354.cel.gz.2,gsm60355.cel.gz.2,gsm60356.cel.gz.2,gsm60357.cel.gz.2,...,gsm60757.cel.gz.2,gsm60758.cel.gz.2,gsm60759.cel.gz.2,gsm60760.cel.gz.2,gsm60761.cel.gz.2,gsm60762.cel.gz.2,gsm60763.cel.gz.2,gsm60764.cel.gz.2,gsm60765.cel.gz.2,gsm60766.cel.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.973291,1.002786,0.960902,0.968952,0.960902,0.955523,0.964991,0.989752,0.964991,0.985918
10,NaN,1.000000,1.000000,1.000000,1.000000,1.000001,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,0.052376,0.056108,0.389558,0.013936,0.271965,0.217711,0.679912,0.857635,0.071383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,0.178932,0.093224,0.833218,0.019740,0.053805,0.034096,0.110334,0.054830,0.061383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,1.000000,1.000001,1.000000,0.999999,0.999999,0.999998,1.000000,1.000000,1.000000,...,0.147233,0.199714,0.975129,0.027550,0.833424,0.026204,0.068082,0.014994,0.067175,0.029489
100009676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.593712,1.388980,1.002786,1.494889,0.695593,0.985918,0.977429,0.996808,0.968952,0.977429
10001,NaN,0.079840,0.006987,0.834696,0.999556,0.196210,3.365996,0.998753,0.042908,0.296908,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,1.000001,1.000000,1.000000,1.000000,0.999999,0.999999,0.999999,1.000001,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,0.999999,1.000000,1.000000,1.000001,1.000000,1.000001,1.000006,0.999999,1.000000,...,0.955523,0.973291,0.955523,0.955540,0.985918,0.975129,0.960902,0.985918,0.977429,0.960902


In [54]:
# outputdf
# gsm_tables_sc500 = gsm_tables
# df_outer_sc500['GSM60349.CEL.gz']
# df_outer['GSM60349.CEL.gz']
# df_outer_sc500
# df_outer
# gsm_tables_sc500['GPL8300']
# gsm_tables['GPL8300']

,GSM60699.CEL.gz,GSM60699.CEL.gz.1,GSM60699.CEL.gz.2,GSM60700.CEL.gz,GSM60700.CEL.gz.1,GSM60700.CEL.gz.2,GSM60701.CEL.gz,GSM60701.CEL.gz.1,GSM60701.CEL.gz.2,GSM60702.CEL.gz,...,GSM60378.CEL.gz.2,GSM60379.CEL.gz,GSM60379.CEL.gz.1,GSM60379.CEL.gz.2,GSM60380.CEL.gz,GSM60380.CEL.gz.1,GSM60380.CEL.gz.2,GSM60381.CEL.gz,GSM60381.CEL.gz.1,GSM60381.CEL.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,33.5,P,0.009985,55.9,P,0.014937,82.0,P,0.001892,50.1,...,0.003067,88.6,P,0.002617,77.0,P,0.001892,80.5,P,0.000959
780 /// 100616237,33.5,P,0.009985,55.9,P,0.014937,82.0,P,0.001892,50.1,...,0.004863,36.9,P,0.000959,48.3,P,0.002617,38.9,P,0.001141
780 /// 100616237,34.1,A,0.129639,32.9,A,0.111572,44.6,P,0.046143,29.1,...,0.003067,88.6,P,0.002617,77.0,P,0.001892,80.5,P,0.000959
780 /// 100616237,34.1,A,0.129639,32.9,A,0.111572,44.6,P,0.046143,29.1,...,0.004863,36.9,P,0.000959,48.3,P,0.002617,38.9,P,0.001141
780 /// 100616237,45.4,P,0.005859,37.6,P,0.010742,44.0,P,0.001953,32.0,...,0.003067,88.6,P,0.002617,77.0,P,0.001892,80.5,P,0.000959
780 /// 100616237,45.4,P,0.005859,37.6,P,0.010742,44.0,P,0.001953,32.0,...,0.004863,36.9,P,0.000959,48.3,P,0.002617,38.9,P,0.001141
780 /// 100616237,52.6,P,0.010742,55.1,P,0.023926,66.8,P,0.002930,63.8,...,0.003067,88.6,P,0.002617,77.0,P,0.001892,80.5,P,0.000959
780 /// 100616237,52.6,P,0.010742,55.1,P,0.023926,66.8,P,0.002930,63.8,...,0.004863,36.9,P,0.000959,48.3,P,0.002617,38.9,P,0.001141
5982,17.7,P,0.039365,14.2,A,0.073830,6.7,A,0.284747,14.6,...,0.013092,20.9,P,0.039365,17.9,M,0.043968,22.9,P,0.021866


In [8]:
type(outputdf)

pandas.core.frame.DataFrame

In [37]:
with localconverter(ro.default_converter + pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(outputdf)

pd_from_r_df

,GSM60349.CEL.gz,GSM60349.CEL.gz.1,GSM60349.CEL.gz.2,GSM60350.CEL.gz,GSM60350.CEL.gz.1,GSM60350.CEL.gz.2,GSM60351.cel.gz,GSM60351.cel.gz.1,GSM60351.cel.gz.2,GSM60352.cel.gz,...,GSM60378.CEL.gz.2,GSM60379.CEL.gz,GSM60379.CEL.gz.1,GSM60379.CEL.gz.2,GSM60380.CEL.gz,GSM60380.CEL.gz.1,GSM60380.CEL.gz.2,GSM60381.CEL.gz,GSM60381.CEL.gz.1,GSM60381.CEL.gz.2
100_g_at,819.696849,M,0.054470,697.976614,P,0.035163,792.199601,M,0.054470,670.927981,...,0.073830,525.182524,M,0.048995,605.096506,P,0.035163,671.219124,P,0.039365
1000_at,666.601875,P,0.011447,377.096623,P,0.035163,633.796783,P,0.019303,369.024441,...,0.014937,502.625967,P,0.014937,575.696476,P,0.027860,476.986010,P,0.009985
1001_at,73.056571,A,0.320830,27.907207,A,0.284747,64.561944,A,0.175989,9.274875,...,0.189687,94.660663,A,0.175989,171.428355,M,0.054470,17.197764,A,0.602006
1002_f_at,11.787246,A,0.926170,53.378871,A,0.358691,57.671637,A,0.541184,14.085050,...,0.520620,19.654731,A,0.765443,22.265423,A,0.715253,5.054754,A,0.983000
1003_s_at,46.570660,A,0.765443,49.144225,A,0.621815,108.366510,A,0.500000,73.856262,...,0.581931,132.972994,A,0.438361,65.490578,A,0.602006,18.326015,A,0.810313
1004_at,276.871626,A,0.218983,350.451574,P,0.011447,252.800586,A,0.127645,212.181074,...,0.204022,157.720347,A,0.189687,215.081926,A,0.081337,156.242030,A,0.234557
1005_at,485.753658,P,0.014937,99.133354,A,0.541184,934.061710,P,0.000266,100.917653,...,0.001354,471.338044,P,0.004863,468.230344,P,0.000468,485.045190,P,0.000468
1006_at,85.230428,A,0.267463,48.180077,A,0.378185,6.745844,A,0.602006,3.240873,...,0.320830,7.598366,A,0.500000,15.661077,A,0.715253,23.027493,A,0.438361
1007_s_at,717.599098,P,0.017000,966.038612,P,0.003067,724.167413,P,0.005643,910.941864,...,0.003067,885.906655,P,0.002617,769.702265,P,0.001892,805.025121,P,0.000959
1008_f_at,1832.103394,P,0.001602,1141.965627,P,0.001141,828.595844,P,0.000805,615.969660,...,0.001602,2256.733954,P,0.000673,2059.737948,P,0.000388,1689.705825,P,0.000219
